In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/vipuser/anaconda3/envs/GLM4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda"
model_path = "/home/LLM_para/para_glm4"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
query = """
You are a SQL table prediction robot. Please return the tables you think are relevant to the question from the database_schema based on the given question. Here is the Q&A template:
#################templete#############################
question:"Show the name and theme for all shows and the number of vocalists in each shows.
database_schema: CREATE TABLE `stadium` (\n  Stadium_ID INT PRIMARY KEY,\n  Location TEXT,\n  Name TEXT,\n  Capacity INT,\n  Highest INT,\n  Lowest INT,\n  Average INT\n);\nSample rows from `stadium`:\n1, Raith Rovers, Stark's Park, 10104, 4812, 1294, 2106\n2, Ayr United, Somerset Park, 11998, 2363, 1057, 1477\n3, East Fife, Bayview Stadium, 2000, 1980, 533, 864\n\nCREATE TABLE `singer` (\n  Singer_ID INT PRIMARY KEY,\n  Name TEXT,\n  Country TEXT,\n  Song_Name TEXT,\n  Song_release_year TEXT,\n  Age INT,\n  Is_male bool\n);\nSample rows from `singer`:\n1, Joe Sharp, Netherlands, You, 1992, 52, F\n2, Timbaland, United States, Dangerous, 2008, 32, T\n3, Justin Brown, France, Hey Oh, 2013, 29, T\n\nCREATE TABLE `concert` (\n  concert_ID INT PRIMARY KEY,\n  concert_Name TEXT,\n  Theme TEXT,\n  Stadium_ID TEXT REFERENCES stadium(Stadium_ID),\n  Year TEXT\n);\nSample rows from `concert`:\n1, Auditions, Free choice, 1, 2014\n2, Super bootcamp, Free choice 2, 2, 2014\n3, Home Visits, Bleeding Love, 2, 2015\n\nCREATE TABLE `singer_in_concert` (\n  concert_ID INT PRIMARY KEY REFERENCES concert(concert_ID),\n  Singer_ID TEXT PRIMARY KEY REFERENCES singer(Singer_ID)\n);\nSample rows from `singer_in_concert`:\n1, 2\n1, 3\n1, 5\n\n
answer: singer_in_concert, concert;
#####################################################
Please generate according to the template rules and return the answer content:
question:{question}
extract_samples:{extract_samples}
answer:
"""

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
instruction = tokenizer(query,add_special_tokens=False )
input_ids = instruction["input_ids"]
print(len(input_ids))

493


In [4]:
inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )
inputs = inputs.to(device)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  8.24it/s]


In [6]:
#  get schame
# 拆解训练集
# 提供每次训练token数
from tqdm import tqdm
import copy 
import pandas as pd
#获取REF表数据
tab_str = ""
count = 0
schema_slicing = []
df = pd.read_csv("/home/code/GLM-4-main/mydataset/table_schema_Reference.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['Reference_group']
    tab_str += table_group + "\n"

# print(ReF_group[5])
# print("#############################")
# print(ReF_group[6])


#获取noREF表数据
df = pd.read_csv("/home/code/GLM-4-main/mydataset/table_schema_noReference.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['noReference_group']
    tab_str += table_group + "\n"

100%|████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 11583.81it/s]


In [7]:
check_str = tab_str + query
check_token = tokenizer(f"{check_str}", add_special_tokens=False)
print(len(check_token["input_ids"]))

10840


In [10]:
from tqdm import tqdm
df = pd.read_csv("/home/code/GLM-4-main/mydataset/validation_dataset_formatted_Myidea3.csv")
results = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    question = row['question']
    correct_tables = row['correct_tables']
    user_message = f"""
You are a SQL table prediction robot. Please return the tables you think are relevant to the question from the database_schema based on the given question. Here is the Q&A template:
#################templete#############################
question:"Show the name and theme for all shows and the number of vocalists in each shows.
database_schema: CREATE TABLE `stadium` (\n  Stadium_ID INT PRIMARY KEY,\n  Location TEXT,\n  Name TEXT,\n  Capacity INT,\n  Highest INT,\n  Lowest INT,\n  Average INT\n);\nSample rows from `stadium`:\n1, Raith Rovers, Stark's Park, 10104, 4812, 1294, 2106\n2, Ayr United, Somerset Park, 11998, 2363, 1057, 1477\n3, East Fife, Bayview Stadium, 2000, 1980, 533, 864\n\nCREATE TABLE `singer` (\n  Singer_ID INT PRIMARY KEY,\n  Name TEXT,\n  Country TEXT,\n  Song_Name TEXT,\n  Song_release_year TEXT,\n  Age INT,\n  Is_male bool\n);\nSample rows from `singer`:\n1, Joe Sharp, Netherlands, You, 1992, 52, F\n2, Timbaland, United States, Dangerous, 2008, 32, T\n3, Justin Brown, France, Hey Oh, 2013, 29, T\n\nCREATE TABLE `concert` (\n  concert_ID INT PRIMARY KEY,\n  concert_Name TEXT,\n  Theme TEXT,\n  Stadium_ID TEXT REFERENCES stadium(Stadium_ID),\n  Year TEXT\n);\nSample rows from `concert`:\n1, Auditions, Free choice, 1, 2014\n2, Super bootcamp, Free choice 2, 2, 2014\n3, Home Visits, Bleeding Love, 2, 2015\n\nCREATE TABLE `singer_in_concert` (\n  concert_ID INT PRIMARY KEY REFERENCES concert(concert_ID),\n  Singer_ID TEXT PRIMARY KEY REFERENCES singer(Singer_ID)\n);\nSample rows from `singer_in_concert`:\n1, 2\n1, 3\n1, 5\n\n
answer: singer_in_concert, concert;
#####################################################
Please generate according to the template rules and return the answer content:
question:{question}
database_schema:{tab_str}
answer:
"""
    inputs = tokenizer.apply_chat_template([{"role": "user", "content": user_message}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )
    inputs = inputs.to(device)
    gen_kwargs = {"max_length": 12000, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"{index}:",response)
    results.append([response,correct_tables])
new_df = pd.DataFrame(results, columns = ['response','correct_tables']) 

  0%|▏                                                                                  | 1/383 [00:04<28:59,  4.55s/it]

0: 
answer: Manufacturers, Products


  1%|▍                                                                                  | 2/383 [00:06<19:06,  3.01s/it]

1: 
answer:
Manufacturers, Products


  1%|▋                                                                                  | 3/383 [00:08<15:42,  2.48s/it]

...

380: 
answer:
railway, train


100%|████████████████████████████████████████████████████████████████████████████████▊| 382/383 [14:56<00:02,  2.06s/it]

381: 
answer:
table: `book_club`
table: `culture_company`


100%|█████████████████████████████████████████████████████████████████████████████████| 383/383 [14:58<00:00,  2.35s/it]

382: 
answer:
 publishers, publications;


In [11]:
new_df.to_csv("generated_table_val_myidea_newset.csv", index=False)